In [44]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

data augmentation

In [45]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset

In [46]:
batch_size = 128
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


Small residual network

In [47]:
class SmallNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,3,3,1,1)
        self.conv2 = nn.Conv2d(3,32,3,1,1)
        self.conv3 = nn.Conv2d(32,64,3,1,1)
        self.conv4 = nn.Conv2d(64,64,3,1,1)
        self.convskip = nn.Conv2d(3,64,3,1,1)
        self.ac = nn.ReLU()
        self.fc = nn.Linear(65536,10)
        

    def forward(self,x):
        x = self.ac(self.conv1(x))
        skip = x
        x = self.ac(self.conv2(x))
        x = self.ac(self.conv3(x))
        skip = self.convskip(skip)
        x = x + skip
        x = self.ac(self.conv4(x))
        x = self.ac(self.conv4(x))
        x = x.view(x.size(0),-1)
        x = self.fc(x)
        return x




In [48]:
device = torch.device("cuda")
model = SmallNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=150)

In [49]:
# Training Function
def train(epoch):
    model.train()
    running_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    print(f"Epoch {epoch}: Loss: {running_loss / len(trainloader):.3f} | Acc: {100. * correct / total:.2f}%")

# Testing Function
def test():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    print(f"Test Accuracy: {100. * correct / total:.2f}%")
    return 100. * correct / total

best_acc = 0
for epoch in range(150):
    train(epoch)
    scheduler.step()
    acc = test()
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), "best_resnet18_cifar10.pth")

    print(f"Best Test Accuracy: {best_acc:.2f}%")


Epoch 0: Loss: 1.729 | Acc: 37.20%
Test Accuracy: 53.23%
Best Test Accuracy: 53.23%
Epoch 1: Loss: 1.342 | Acc: 52.03%
Test Accuracy: 60.58%
Best Test Accuracy: 60.58%
Epoch 2: Loss: 1.171 | Acc: 58.59%
Test Accuracy: 64.90%
Best Test Accuracy: 64.90%
Epoch 3: Loss: 1.082 | Acc: 61.71%
Test Accuracy: 66.59%
Best Test Accuracy: 66.59%
Epoch 4: Loss: 1.018 | Acc: 64.17%
Test Accuracy: 69.09%
Best Test Accuracy: 69.09%
Epoch 5: Loss: 0.965 | Acc: 65.98%
Test Accuracy: 70.60%
Best Test Accuracy: 70.60%
Epoch 6: Loss: 0.928 | Acc: 67.49%
Test Accuracy: 71.33%
Best Test Accuracy: 71.33%
Epoch 7: Loss: 0.896 | Acc: 68.63%
Test Accuracy: 72.12%
Best Test Accuracy: 72.12%
Epoch 8: Loss: 0.866 | Acc: 69.63%
Test Accuracy: 72.87%
Best Test Accuracy: 72.87%
Epoch 9: Loss: 0.848 | Acc: 70.37%
Test Accuracy: 73.04%
Best Test Accuracy: 73.04%
Epoch 10: Loss: 0.818 | Acc: 71.57%
Test Accuracy: 72.72%
Best Test Accuracy: 73.04%
Epoch 11: Loss: 0.797 | Acc: 72.16%
Test Accuracy: 73.92%
Best Test Accurac

Custom residual network

In [54]:
class CostumNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,32,3,1,1)
        self.conv2 = nn.Conv2d(32,64,3,1,1)
        self.conv3 = nn.Conv2d(64,128,3,1,1)
        self.conv4 = nn.Conv2d(128,128,3,1,1)
        self.ac1 = nn.ReLU()
        self.ac2 = nn.Softmax()
        self.fc = nn.Linear(131072,10)
        self.convskip1 = nn.Conv2d(32,128,1,1)

    def forward(self,x):
        x = self.ac1(self.conv1(x))
        skip = x
        x = self.ac2(self.conv2(x))
        x = self.ac2(self.conv3(x))
        skip = self.convskip1(skip)
        x = x + skip
        x = self.ac1(self.conv4(x))
        x = x.view(x.size(0),-1)
        x = self.fc(x)
        return x


In [55]:
device = torch.device("cuda")
model = CostumNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=150)

In [56]:
best_acc = 0
for epoch in range(150):
    train(epoch)
    scheduler.step()
    acc = test()
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), "best_resnet18_cifar10.pth")

    print(f"Best Test Accuracy: {best_acc:.2f}%")

/tmp/ipykernel_10181/1302730542.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.ac2(self.conv2(x))
/tmp/ipykernel_10181/1302730542.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.ac2(self.conv3(x))


Epoch 0: Loss: 1.844 | Acc: 33.50%
Test Accuracy: 40.48%
Best Test Accuracy: 40.48%
Epoch 1: Loss: 1.641 | Acc: 40.08%
Test Accuracy: 44.66%
Best Test Accuracy: 44.66%
Epoch 2: Loss: 1.581 | Acc: 42.21%
Test Accuracy: 48.80%
Best Test Accuracy: 48.80%
Epoch 3: Loss: 1.521 | Acc: 44.52%
Test Accuracy: 49.21%
Best Test Accuracy: 49.21%
Epoch 4: Loss: 1.470 | Acc: 46.10%
Test Accuracy: 53.24%
Best Test Accuracy: 53.24%
Epoch 5: Loss: 1.413 | Acc: 48.87%
Test Accuracy: 55.71%
Best Test Accuracy: 55.71%
Epoch 6: Loss: 1.376 | Acc: 50.39%
Test Accuracy: 55.27%
Best Test Accuracy: 55.71%
Epoch 7: Loss: 1.335 | Acc: 51.93%
Test Accuracy: 56.97%
Best Test Accuracy: 56.97%
Epoch 8: Loss: 1.314 | Acc: 52.87%
Test Accuracy: 58.04%
Best Test Accuracy: 58.04%
Epoch 9: Loss: 1.287 | Acc: 53.89%
Test Accuracy: 60.70%
Best Test Accuracy: 60.70%
Epoch 10: Loss: 1.230 | Acc: 56.26%
Test Accuracy: 61.12%
Best Test Accuracy: 61.12%
Epoch 11: Loss: 1.187 | Acc: 57.88%
Test Accuracy: 61.70%
Best Test Accurac

Pre-trained ResNet

In [57]:
from torchvision.models import resnet18
model = resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)  # Adjust final layer for 10 CIFAR-10 classes

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=150)

In [58]:
best_acc = 0
for epoch in range(150):
    train(epoch)
    scheduler.step()
    acc = test()
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), "best_resnet18_cifar10.pth")

    print(f"Best Test Accuracy: {best_acc:.2f}%")

Epoch 0: Loss: 1.031 | Acc: 64.65%
Test Accuracy: 72.41%
Best Test Accuracy: 72.41%
Epoch 1: Loss: 0.744 | Acc: 74.73%
Test Accuracy: 76.90%
Best Test Accuracy: 76.90%
Epoch 2: Loss: 0.651 | Acc: 77.81%
Test Accuracy: 79.14%
Best Test Accuracy: 79.14%
Epoch 3: Loss: 0.590 | Acc: 79.88%
Test Accuracy: 77.94%
Best Test Accuracy: 79.14%
Epoch 4: Loss: 0.568 | Acc: 80.49%
Test Accuracy: 80.56%
Best Test Accuracy: 80.56%
Epoch 5: Loss: 0.518 | Acc: 82.12%
Test Accuracy: 81.17%
Best Test Accuracy: 81.17%
Epoch 6: Loss: 0.487 | Acc: 83.26%
Test Accuracy: 81.54%
Best Test Accuracy: 81.54%
Epoch 7: Loss: 0.475 | Acc: 83.82%
Test Accuracy: 81.93%
Best Test Accuracy: 81.93%
Epoch 8: Loss: 0.445 | Acc: 84.69%
Test Accuracy: 82.62%
Best Test Accuracy: 82.62%
Epoch 9: Loss: 0.428 | Acc: 85.27%
Test Accuracy: 82.78%
Best Test Accuracy: 82.78%
Epoch 10: Loss: 0.411 | Acc: 85.83%
Test Accuracy: 83.82%
Best Test Accuracy: 83.82%
Epoch 11: Loss: 0.403 | Acc: 86.07%
Test Accuracy: 82.95%
Best Test Accurac

Inception-style network

In [59]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Inception(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1x1 = nn.Conv2d(3,64,1)
        self.conv3x3 = nn.Conv2d(3,128,3,1,1)
        self.conv5x5 = nn.Conv2d(3,32,5,1,2)
        self.maxpool = nn.MaxPool2d(3,1,1)
        self.conv_pool = nn.Conv2d(3,32,1)
        self.fc = nn.Linear(64 + 128 + 32 + 32, 10)

    def forward(self, x):
        x1 = self.conv1x1(x)
        x1 = F.relu(x1)

        x2 = self.conv3x3(x)
        x2 = F.relu(x2)

        x3 = self.conv5x5(x)
        x3 = F.relu(x3)

        x4 = self.maxpool(x)
        x4 = self.conv_pool(x4)
        x4 = F.relu(x4)

        x = torch.cat([x1, x2, x3, x4], 1)

        x = F.adaptive_avg_pool2d(x, (1, 1))  
        x = x.view(x.size(0), -1)  

        x = self.fc(x)

        return x




In [60]:
device = torch.device("cuda")
model = Inception().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=150)

In [61]:
best_acc = 0
for epoch in range(150):
    train(epoch)
    scheduler.step()
    acc = test()
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), "best_resnet18_cifar10.pth")

    print(f"Best Test Accuracy: {best_acc:.2f}%")

Epoch 0: Loss: 2.073 | Acc: 24.02%
Test Accuracy: 27.28%
Best Test Accuracy: 27.28%
Epoch 1: Loss: 1.929 | Acc: 29.58%
Test Accuracy: 30.43%
Best Test Accuracy: 30.43%
Epoch 2: Loss: 1.866 | Acc: 30.89%
Test Accuracy: 30.28%
Best Test Accuracy: 30.43%
Epoch 3: Loss: 1.833 | Acc: 31.75%
Test Accuracy: 32.34%
Best Test Accuracy: 32.34%
Epoch 4: Loss: 1.809 | Acc: 32.69%
Test Accuracy: 33.19%
Best Test Accuracy: 33.19%
Epoch 5: Loss: 1.792 | Acc: 33.35%
Test Accuracy: 32.64%
Best Test Accuracy: 33.19%
Epoch 6: Loss: 1.777 | Acc: 33.93%
Test Accuracy: 34.14%
Best Test Accuracy: 34.14%
Epoch 7: Loss: 1.763 | Acc: 34.90%
Test Accuracy: 34.60%
Best Test Accuracy: 34.60%
Epoch 8: Loss: 1.752 | Acc: 35.07%
Test Accuracy: 35.12%
Best Test Accuracy: 35.12%
Epoch 9: Loss: 1.743 | Acc: 35.58%
Test Accuracy: 35.45%
Best Test Accuracy: 35.45%
Epoch 10: Loss: 1.733 | Acc: 35.92%
Test Accuracy: 36.18%
Best Test Accuracy: 36.18%
Epoch 11: Loss: 1.725 | Acc: 36.15%
Test Accuracy: 36.76%
Best Test Accurac

In [62]:
class ResNet18(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet18, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        
        # Manually define layers for ResNet-18
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(64)
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(128)
        
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn4 = nn.BatchNorm2d(128)
        
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn5 = nn.BatchNorm2d(256)
        
        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn6 = nn.BatchNorm2d(256)
        
        self.conv7 = nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn7 = nn.BatchNorm2d(512)
        
        self.conv8 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn8 = nn.BatchNorm2d(512)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))
        x = torch.relu(self.bn2(self.conv2(x)))
        x = torch.relu(self.bn3(self.conv3(x)))
        x = torch.relu(self.bn4(self.conv4(x)))
        x = torch.relu(self.bn5(self.conv5(x)))
        x = torch.relu(self.bn6(self.conv6(x)))
        x = torch.relu(self.bn7(self.conv7(x)))
        x = torch.relu(self.bn8(self.conv8(x)))

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x


In [63]:
model = ResNet18().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=150)

In [64]:
best_acc = 0
for epoch in range(150):
    train(epoch)
    scheduler.step()
    acc = test()
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), "best_resnet18_cifar10.pth")

    print(f"Best Test Accuracy: {best_acc:.2f}%")

Epoch 0: Loss: 1.631 | Acc: 39.57%
Test Accuracy: 49.25%
Best Test Accuracy: 49.25%
Epoch 1: Loss: 1.147 | Acc: 58.50%
Test Accuracy: 57.52%
Best Test Accuracy: 57.52%
Epoch 2: Loss: 0.905 | Acc: 68.13%
Test Accuracy: 65.79%
Best Test Accuracy: 65.79%
Epoch 3: Loss: 0.757 | Acc: 73.41%
Test Accuracy: 70.97%
Best Test Accuracy: 70.97%
Epoch 4: Loss: 0.668 | Acc: 76.82%
Test Accuracy: 71.16%
Best Test Accuracy: 71.16%
Epoch 5: Loss: 0.613 | Acc: 78.76%
Test Accuracy: 70.28%
Best Test Accuracy: 71.16%
Epoch 6: Loss: 0.572 | Acc: 80.32%
Test Accuracy: 78.30%
Best Test Accuracy: 78.30%
Epoch 7: Loss: 0.541 | Acc: 81.24%
Test Accuracy: 76.04%
Best Test Accuracy: 78.30%
Epoch 8: Loss: 0.518 | Acc: 82.18%
Test Accuracy: 71.17%
Best Test Accuracy: 78.30%
Epoch 9: Loss: 0.505 | Acc: 82.47%
Test Accuracy: 78.28%
Best Test Accuracy: 78.30%
Epoch 10: Loss: 0.489 | Acc: 83.10%
Test Accuracy: 73.77%
Best Test Accuracy: 78.30%
Epoch 11: Loss: 0.480 | Acc: 83.45%
Test Accuracy: 79.87%
Best Test Accurac